In [1]:
import nest_asyncio
import numpy as np
import tensorflow as tf

from ocddetection import models
from ocddetection.federated.learning import stateful
from ocddetection.federated.learning.impl.personalization import mixed

/home/Lando.Loeper/miniconda3/envs/fl/lib/python3.8/site-packages/tensorflow_federated/python/core/api/computation_types.py:492: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  py_typecheck.check_type(elements, collections.Iterable)
/home/Lando.Loeper/miniconda3/envs/fl/lib/python3.8/site-packages/tensorflow_federated/python/common_libs/py_typecheck.py:177: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if not isinstance(element, collections.Sequence) or len(element) != 2:
/home/Lando.Loeper/miniconda3/envs/fl/lib/python3.8/site-packages/tensorflow_federated/python/core/impl/executors/caching_executor.py:33: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 i

In [2]:
nest_asyncio.apply()

/home/Lando.Loeper/miniconda3/envs/fl/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model = models.bidirectional(5, 2, 1, 8, 0.5)

In [ ]:
model.summary()

In [ ]:
model.trainable_variables

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='loss')
train_metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy')]

In [ ]:
def __train_step(X, y, model, optimizer, loss_fn, loss, metrics):
    with tf.GradientTape() as tape:
        y_hat = model(X, training=True)
        _loss = loss_fn(y, y_hat)

    optimizer.apply_gradients(
        zip(
            tape.gradient(_loss, model.trainable_variables),
            model.trainable_variables
        )
    )

    loss(_loss)
    for metric in metrics:
        metric(y, y_hat)

def __log(key: str, epoch: int, weight: int, value: float, state):
    state[key][epoch].append((weight, value))

In [ ]:
train_fn = tf.function(
    partial(
        __train_step,
        model=model,
        optimizer=optimizer,
        loss_fn=loss_fn,
        loss=train_loss,
        metrics=train_metrics
    )
)

In [ ]:
num_examples = tf.Variable(0, trainable=False, dtype=tf.int32)

In [ ]:
logger = defaultdict(lambda: defaultdict(list))

In [ ]:
log = partial(__log, state=logger)

In [ ]:
for epoch in range(1, 6):
    num_examples.assign(0)
    
    train_loss.reset_states()
    for metric in train_metrics:
        metric.reset_states()
    
    for X, y in data:
        train_fn(X, y)
        num_examples.assign_add(tf.shape(y)[0])
    
    log(train_loss.name, epoch, num_examples.read_value().numpy(), train_loss.result().numpy())
    for metric in train_metrics:
        log(metric.name, epoch, num_examples.read_value().numpy(), metric.result().numpy())

In [ ]:
logger